In [190]:
# 데이터 합치기
import pandas as pd

df_rsna = pd.read_csv('./data/rsna/stage_2_detailed_class_info.csv')
df_mimic = pd.read_csv('./data/mimic-cxr/mimic-cxr-2.0.0-chexpert.csv.gz', compression='gzip')
df_mimic_record = pd.read_csv('cxr-record-list.csv.gz', compression='gzip')
df_mimic_meta = pd.read_csv('mimic-cxr-2.0.0-metadata.csv.gz', compression='gzip')
df_mimic_list = pd.read_csv('cxr-study-list.csv.gz', compression='gzip')

In [50]:
df_mimic_meta.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,ViewCodeSequence_CodeMeaning,PatientOrientationCodeSequence_CodeMeaning
0,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,10000032,50414267,CHEST (PA AND LAT),PA,3056,2544,21800506,213014.531,CHEST (PA AND LAT),postero-anterior,Erect
1,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,10000032,50414267,CHEST (PA AND LAT),LATERAL,3056,2544,21800506,213014.531,CHEST (PA AND LAT),lateral,Erect
2,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,10000032,53189527,CHEST (PA AND LAT),PA,3056,2544,21800626,165500.312,CHEST (PA AND LAT),postero-anterior,Erect
3,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,10000032,53189527,CHEST (PA AND LAT),LATERAL,3056,2544,21800626,165500.312,CHEST (PA AND LAT),lateral,Erect
4,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,10000032,53911762,CHEST (PORTABLE AP),AP,2705,2539,21800723,80556.875,CHEST (PORTABLE AP),antero-posterior,NaN


## RSNA 데이터셋 정리

| df_rsna | df |
|---|---|
|patientId|file_name|
|class|class|
|-|patient_id|
|-|study_id|

In [10]:
df_rsna

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In [189]:
rsna_tmp = pd.DataFrame()

rsna_tmp['patient_id'] = None
rsna_tmp['study_id'] = None
rsna_tmp['class'] = df_rsna['class'].map({
    'Normal': 0,
    'Lung Opacity': 1,
    'No Lung Opacity / Not Normal': 2
})
rsna_tmp['file_name'] = df_rsna['patientId']+'.jpg'

rsna = rsna_tmp.groupby('file_name').first().reset_index().copy()
rsna

,file_name,patient_id,study_id,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6.jpg,None,None,2
1,000924cf-0f8d-42bd-9158-1af53881a557.jpg,None,None,0
2,000db696-cf54-4385-b10b-6b16fbb3f985.jpg,None,None,1
3,000fe35a-2649-43d4-b027-e67796d412e0.jpg,None,None,1
4,001031d9-f904-4a23-b3e5-2c088acd19c6.jpg,None,None,1
...,...,...,...,...
26679,fffb2395-8edd-4954-8a89-ffe2fd329be3.jpg,None,None,1
26680,fffba05a-1635-4545-9bbd-57ad4cfe8d27.jpg,None,None,0
26681,fffc95b5-605b-4226-80ab-62caec682b22.jpg,None,None,2
26682,fffcff11-d018-4414-971a-a7cefa327795.jpg,None,None,2


In [114]:
import numpy as np

nan_mask_1 = rsna['patient_id'].isna()
n_missing = nan_mask_1.sum()
new_ids = np.arange(20000001, 20000001 + n_missing)
rsna.loc[nan_mask_1, 'patient_id'] = new_ids

nan_mask_2 = rsna['study_id'].isna()
n_missing = nan_mask_2.sum()
new_ids = np.arange(60000001, 60000001 + n_missing)
rsna.loc[nan_mask_2, 'study_id'] = new_ids

In [137]:
rsna['set_name'] = 'rsna'
rsna

,file_name,patient_id,study_id,class,set_name
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,20000001,60000001,2,rsna
1,000924cf-0f8d-42bd-9158-1af53881a557,20000002,60000002,0,rsna
2,000db696-cf54-4385-b10b-6b16fbb3f985,20000003,60000003,1,rsna
3,000fe35a-2649-43d4-b027-e67796d412e0,20000004,60000004,1,rsna
4,001031d9-f904-4a23-b3e5-2c088acd19c6,20000005,60000005,1,rsna
...,...,...,...,...,...
26679,fffb2395-8edd-4954-8a89-ffe2fd329be3,20026680,60026680,1,rsna
26680,fffba05a-1635-4545-9bbd-57ad4cfe8d27,20026681,60026681,0,rsna
26681,fffc95b5-605b-4226-80ab-62caec682b22,20026682,60026682,2,rsna
26682,fffcff11-d018-4414-971a-a7cefa327795,20026683,60026683,2,rsna


In [116]:
print((rsna['class']==0).sum())
print((rsna['class']==1).sum())
print((rsna['class']==2).sum())

8851
6012
11821


## MIMIC 데이터셋 정리

| df_mimic + mimic_record | df |
|---|---|
|subject_id|patient_id|
|study_id|study_id|
|class|Pneumonia|
|dicom_id|file_name|

**dicom_id 중에서 PA 만 dataset으로 사용

In [186]:
mimic = pd.DataFrame()

pneumonia_cases = df_mimic[df_mimic['Pneumonia']==1.0].copy()
only_pa_cases = df_mimic_meta[df_mimic_meta['ViewPosition']=='PA'].copy()

pneumonia_pa_cases = pd.merge(
    pneumonia_cases[['subject_id','study_id', 'Pneumonia']],
    only_pa_cases[['dicom_id', 'study_id', 'ViewPosition']],
    on='study_id',
    how='inner'
)

mimic['patient_id'] = pneumonia_pa_cases['subject_id']
mimic['study_id'] = pneumonia_pa_cases['study_id']
mimic['class'] = pneumonia_pa_cases['Pneumonia'].map({
    1.0: 1
})
mimic['file_name'] = pneumonia_pa_cases['dicom_id']+'.jpg'

mimic['set_name'] = 'mimic'


In [187]:
mimic

,patient_id,study_id,class,file_name,set_name
0,10001884,50807032,1,ebf48d65-7e780cd5-59118fba-50977097-3720cc7e.jpg,mimic
1,10003019,50543252,1,3f4a324f-7967a6b4-91edf0c8-94fbefd4-32402065.jpg,mimic
2,10003019,58505074,1,9f0a463b-dbbe3f8e-d5cf30af-9452e738-5465e7bf.jpg,mimic
3,10004235,54234360,1,2af702f0-3c2b86f3-82e2112b-7f449f3d-dde1c122.jpg,mimic
4,10004235,54234360,1,8b2702ec-e57c39b6-46a6f813-5cad5e2b-10594109.jpg,mimic
...,...,...,...,...,...
5828,19994730,55555437,1,4ad13d49-1ec820d0-bf9d36a7-9a294c4d-ad34f6c1.jpg,mimic
5829,19994730,55555437,1,703f9b63-3eaf5939-1f83819e-f7b21c76-fe6f2cce.jpg,mimic
5830,19994730,58059795,1,0657a92b-c9c57330-8991578f-a2e1dcff-66f10ac8.jpg,mimic
5831,19994772,55300754,1,1e67c56b-b4328181-a35b4cd3-fd792b54-aac11d32.jpg,mimic


In [166]:
print((mimic['class']==0).sum())
print((mimic['class']==1).sum())
print((mimic['class']==2).sum())

0
5833
0


## NIH 데이터셋 정리

| df_nih | df |
|---|---|
|subject_id|patient_id|
|study_id|study_id|
|class|Pneumonia|
|dicom_id|file_name|

**dicom_id 중에서 PA 만 dataset으로 사용

In [167]:
df_nih = pd.read_csv('./data/nih/Data_Entry_2017.csv')
df_nih['study_id'] = df_nih['Patient ID'].astype('str') + df_nih['Follow-up #'].astype('str')
df_nih

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,study_id
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,10
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,11
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,12
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,20
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN,308011
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN,308020
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN,308030
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN,308040


In [168]:
df_nih_normal = df_nih[(df_nih['View Position']=='PA')&(df_nih['Finding Labels']=='No Finding')]
df_nih_normal

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,study_id
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,20
13,00000005_000.png,No Finding,0,5,69,F,PA,2048,2500,0.168,0.168,NaN,50
16,00000005_003.png,No Finding,3,5,69,F,PA,2992,2991,0.143,0.143,NaN,53
17,00000005_004.png,No Finding,4,5,70,F,PA,2986,2991,0.143,0.143,NaN,54
18,00000005_005.png,No Finding,5,5,70,F,PA,2514,2991,0.143,0.143,NaN,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112114,00030801_000.png,No Finding,0,30801,39,M,PA,2500,2048,0.168,0.168,NaN,308010
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN,308020
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN,308030
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN,308040


In [177]:
nih_subset = pd.DataFrame()
nih_subset['patient_id'] = df_nih_normal['Patient ID']
nih_subset['study_id'] = df_nih_normal['study_id']
nih_subset['class'] = 0
nih_subset['set_name'] = 'nih'
nih_subset['file_name'] = df_nih_normal['Image Index']

df_sample = nih_subset.sample(n=4000, random_state=42).copy()
df_sample

,patient_id,study_id,class,set_name,file_name
80773,19856,198560,0,nih,00019856_000.png
3742,1020,10200,0,nih,00001020_000.png
31370,8187,81871,0,nih,00008187_001.png
12787,3360,33603,0,nih,00003360_003.png
58064,14364,143640,0,nih,00014364_000.png
...,...,...,...,...,...
359,90,9010,0,nih,00000090_010.png
56418,14032,140328,0,nih,00014032_008.png
12682,3328,33280,0,nih,00003328_000.png
73786,18144,181447,0,nih,00018144_007.png


In [178]:
df_nih_pneumonia = df_nih[(df_nih['View Position']=='PA') & (df_nih['Finding Labels'].str.contains('Pneumonia'))]
df_nih_pneumonia

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,study_id
253,00000056_000.png,Nodule|Pneumonia,0,56,76,M,PA,2500,2048,0.168000,0.168000,NaN,560
640,00000165_001.png,Pneumonia,1,165,76,M,PA,2992,2991,0.143000,0.143000,NaN,1651
901,00000218_000.png,Infiltration|Pneumonia,0,218,33,M,PA,2500,2048,0.171000,0.171000,NaN,2180
902,00000218_001.png,Pneumonia,1,218,33,M,PA,2048,2500,0.171000,0.171000,NaN,2181
1078,00000261_007.png,Infiltration|Pneumonia,7,261,41,F,PA,2992,2991,0.143000,0.143000,NaN,2617
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109515,00029808_003.png,Cardiomegaly|Consolidation|Effusion|Pneumonia,3,29808,32,M,PA,2021,2021,0.194311,0.194311,NaN,298083
109877,00029889_000.png,Pneumonia,0,29889,44,F,PA,2021,2021,0.194311,0.194311,NaN,298890
110916,00030249_001.png,Infiltration|Pneumonia,1,30249,64,M,PA,2021,2021,0.194311,0.194311,NaN,302491
111285,00030398_000.png,Consolidation|Effusion|Pneumonia,0,30398,55,F,PA,1684,2021,0.194311,0.194311,NaN,303980


In [179]:
nih_subset = pd.DataFrame()
nih_subset['patient_id'] = df_nih_pneumonia['Patient ID']
nih_subset['study_id'] = df_nih_pneumonia['study_id']
nih_subset['class'] = 1
nih_subset['set_name'] = 'nih'
nih_subset['file_name'] = df_nih_pneumonia['Image Index']

In [180]:
nih_subset = pd.concat([nih_subset, df_sample])
nih_subset

,patient_id,study_id,class,set_name,file_name
253,56,560,1,nih,00000056_000.png
640,165,1651,1,nih,00000165_001.png
901,218,2180,1,nih,00000218_000.png
902,218,2181,1,nih,00000218_001.png
1078,261,2617,1,nih,00000261_007.png
...,...,...,...,...,...
359,90,9010,0,nih,00000090_010.png
56418,14032,140328,0,nih,00014032_008.png
12682,3328,33280,0,nih,00003328_000.png
73786,18144,181447,0,nih,00018144_007.png


## 데이터 합치기

In [181]:
combined_df = pd.concat([rsna, mimic,nih_subset], ignore_index=True)
combined_df

,file_name,patient_id,study_id,class,set_name
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,20000001,60000001,2,rsna
1,000924cf-0f8d-42bd-9158-1af53881a557,20000002,60000002,0,rsna
2,000db696-cf54-4385-b10b-6b16fbb3f985,20000003,60000003,1,rsna
3,000fe35a-2649-43d4-b027-e67796d412e0,20000004,60000004,1,rsna
4,001031d9-f904-4a23-b3e5-2c088acd19c6,20000005,60000005,1,rsna
...,...,...,...,...,...
37142,00000090_010.png,90,9010,0,nih
37143,00014032_008.png,14032,140328,0,nih
37144,00003328_000.png,3328,33280,0,nih
37145,00018144_007.png,18144,181447,0,nih


In [182]:
print((combined_df['class']==0).sum())
print((combined_df['class']==1).sum())
print((combined_df['class']==2).sum())

12851
12475
11821


In [ ]:
combined_df.to_csv('./dataset/cxr-pneumonia-dataset.csv')

In [194]:
from split import split_dataset

train_df, val_df, test_df = split_dataset('cxr-pneumonia-dataset.csv')

# 각 데이터프레임에 split 정보를 담은 컬럼 추가 후 combined_df로 병합
combined_df = pd.concat([
    train_df.assign(split='train'),
    val_df.assign(split='val'),
    test_df.assign(split='test')
], ignore_index=True)
combined_df.to_csv('./dataset/dataset.csv')

# split dataframe 저장
train_df.to_csv('./dataset/train.csv')
val_df.to_csv('./dataset/val.csv')
test_df.to_csv('./dataset/test.csv')

Successfully loaded dataset with 37147 records.

--- Split Summary ---
Total data:      37147
Train data:      29717 (80.00%)
Validation data: 3715 (10.00%)
Test data:       3715 (10.00%)

--- Train Set Distribution ---
set_name  class
mimic     1        0.1570
nih       0        0.1077
          1        0.0170
rsna      0        0.2383
          1        0.1618
          2        0.3182
dtype: float64

--- Validation Set Distribution ---
set_name  class
mimic     1        0.1569
nih       0        0.1077
          1        0.0170
rsna      0        0.2382
          1        0.1620
          2        0.3182
dtype: float64

--- Test Set Distribution ---
set_name  class
mimic     1        0.1572
nih       0        0.1077
          1        0.0170
rsna      0        0.2382
          1        0.1618
          2        0.3182
dtype: float64
